# 7.g Modele encoder-decoder i mechanizm uwagi

Zbudujemy model służący to tłumaczenia krótkich zdań w języku polskim na język angielski. Model będzie oparty o architekturę encoder-decoder, wykorzystującą mechanizm uwagi. Warstwy rekurencyjne to warstwy GRU.
Dane pochodzą z korpusu Open Subtitles: http://opus.nlpl.eu/OpenSubtitles-v2018.php Dane poddano selekcji i filtrowaniu.

Notatnik jest oparty o tutorial do budowania chatbotów którego autorem jest Matthew Inkawhich, umieszczony na stronie pytorcha: https://pytorch.org/tutorials/beginner/chatbot_tutorial.html,

In [ ]:
import torch
DEVICE = torch.device("cuda:0")

Pobranie danych

In [ ]:
!wget https://github.com/sagespl/nlp-masterclass/blob/main/modu%C5%82-07/encoder.model?raw=true
!mv encoder.model?raw=true encoder.model

!wget https://github.com/sagespl/nlp-masterclass/blob/main/modu%C5%82-07/decoder.model?raw=true
!mv decoder.model?raw=true decoder.model

!wget https://github.com/sagespl/nlp-masterclass/blob/main/modu%C5%82-07/seq2seq_data.zip?raw=true
!mv seq2seq_data.zip?raw=true seq2seq_data.zip
!unzip seq2seq_data.zip

In [ ]:
import json


with open("seq2seq_data/data.json") as f:
    data = json.load(f)

with open("seq2seq_data/pl_voc.json") as f: # korzystamy z gotowego słownika
    pl_voc = json.load(f) # 15000 kluczy

with open("seq2seq_data/en_voc.json") as f:
    en_voc = json.load(f) # 10000 kluczy

Rzut oka na dane

In [ ]:
print(pl_voc[:10])
print(data[0])

#### Definicja funkcji służacej do przygotowywania danych

In [ ]:
from torch import Tensor


MAXLEN = 10 +1 # maksymalna długość sekwencji w danych + token EOS
# definiujemy tokeny specjalne
PL_PAD_TOKEN = len(pl_voc) + 0
PL_EOS_TOKEN = len(pl_voc) + 1

EN_PAD_TOKEN = len(en_voc) + 0
EN_SOS_TOKEN = len(en_voc) + 1 # token oznaczający początek sekwencji
EN_EOS_TOKEN = len(en_voc) + 2 # token oznaczający koniec sekwencji

def examples_to_batch(examples, maxlen):
    pl_ids, en_ids = [], []
    for pl, en in examples:
        pl_ids.append([w for w in pl]) # kopiowanie list idków
        en_ids.append([w for w in en] + [EN_EOS_TOKEN]) # kopiowanie list idków + dodanie tokenu oznaczającego koniec sekwencji

    len_indices = [(i, len(toks)) for i, toks in list(enumerate(pl_ids))] 
    len_indices = sorted(len_indices, key=lambda x:x[1], reverse=True) 
    pl_ids = [pl_ids[i[0]] for i in len_indices] # sortowanie inputu i targetu zgodnie z długością inputu
    en_ids = [en_ids[i[0]] for i in len_indices] # na potrzeby funkcji pack_padded_sequence
    
    pl_lens = torch.LongTensor([len(pl) for pl in pl_ids]) # długości sekwencji (przed paddingiem)
    en_lens = torch.LongTensor([len(en) for en in en_ids])
    pl_maxlen = max(pl_lens)
    en_maxlen = max(en_lens)
    for pl in pl_ids: # padding danych treningowych do jednej długości
        while len(pl) < pl_maxlen:
            pl.append(PL_PAD_TOKEN)
    for en in en_ids:
        while len(en) < en_maxlen:
            en.append(EN_PAD_TOKEN)
    Y_mask = [[int(x != EN_PAD_TOKEN) for x in y] for y in en_ids] # maska dla paddingu
    Y_mask = torch.BoolTensor(Y_mask).transpose(1,0) # wszystkie tensory będą miały kształt (liczba tokenów, liczba sekwencji w batchu)
    X = torch.LongTensor(pl_ids).transpose(1,0)
    Y = torch.LongTensor(en_ids).transpose(1,0)
    return X, Y, pl_lens, en_lens, Y_mask 
    # zwracamy: wejście do enkodera, wzorcowe wyjście z dekodera, długość sekwencji dla enkodera, i dekodera (obie przed paddingiem), maskę paddingu dla dekodera

#### Konstrukcja sieci

In [ ]:
from torch import nn

class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, voc_size, n_layers=1, dropout=0): 
        # rozmiar wewnętrznej reprezentacji, wielkośc słownika, liczba warstw rekurencyjnych, dropout części rekurencyjnej
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers # część rekurencyjna może mieć kilka warstw
        self.hidden_size = hidden_size # rozmiar wewnętrznych reprezentacji
        self.embedding = nn.Embedding(voc_size, hidden_size) # warstwa zanurzająca, uczy się kompresować reprezentację one-hot do wektora o długości hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, # gru - mniejszy kuzyn LSTM,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True) # wykorzystujemy wariant dwukierunkowy


    def forward(self, input_seq, input_lengths, hidden=None):
        embedded = self.embedding(input_seq) # konwersja indeksów słów w wektory
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)# pakowanie batcha danych dla warstwy GRU
        outputs, hidden = self.gru(packed, hidden)
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs) # odpakowywanie wyjścia z warstwy GRU
        left_to_right = outputs[:, :, :self.hidden_size]
        right_to_left = outputs[:, : ,self.hidden_size:]
        outputs =  left_to_right + right_to_left  # sumujemy reprezentacje z obu kierunków warstwy rekurencyjnej
        return outputs, hidden # zwracamy wyjście i stan ukryty

In [ ]:
class Attn(nn.Module): # warstwa uwagi oparta na artykule Luonga et al.
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method # wybieramy metodę liczenia uwagi

        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size)) # warstwa bez biasu
        self.softmax = nn.Softmax(dim=1)

    def dot_score(self, hidden, encoder_output):
        # dot operuje na zasadzie iloczynu skalarnego stanu ukrytego enkodera i stanu ukrytego dekodera
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        # general wykorzystuje iloczyn skalarny stanu ukrytego dekodera i outputu z warstwy neuronów która przyjmuje stan ukryty enkodera
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        # concat wykorzystuje warstwę neuronów która przyjmuje konkatenację stanu ukrytego enkodera
        #   i stanu ukrytego dekodera. Po przejściu przez warstwę aplikowana jest funkcja aktywacji
        #   tanh, następnie output przechodzi przez kolejną warstwę neuronów, bez biasu (nn.Parameter)
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # obliczanie wag uwagi na podstawie stanu ukrytego dekodera, i stanów ukrytych enkodera
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)


        attn_energies = attn_energies.t() # transpozycja

        # zwracamy wagi normalizowane przez softmax
        return self.softmax(attn_energies).unsqueeze(1)

In [ ]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, hidden_size, voc_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        self.attn_model = attn_model # jedna z trzech metod liczenia uwagi
        self.hidden_size = hidden_size # rozmiar reprezentacji wewnętrznych
        self.output_size = voc_size # wyjście - reprezentacje one-hot słów angielskich
        self.n_layers = n_layers # dekoder może być wielowarstwowy
        self.dropout = dropout # dropout dla unikania przeuczania

        # Define layers
        self.embedding = nn.Embedding(voc_size, hidden_size) # warstwa zanurzająca do reprezentacji inputu (słów zwracanych przez sam dekoder)
        self.embedding_dropout = nn.Dropout(dropout) # dropout po warstwie zanurzającej
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size) # warstwa przetwarzająca
        self.out = nn.Linear(hidden_size, voc_size) # klasyfikator
        self.softmax = nn.Softmax(dim=1)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # dekoder przechodzi przez cały batch słowo po słowie (i.e. zbiera najpierw wszystkie pierwsze słowa, potem wszystkie drugie itd.)
        embedded = self.embedding(input_step) # zanurzenie  słów
        embedded = self.embedding_dropout(embedded) # aplikacja dropoutu
        rnn_output, hidden = self.gru(embedded, last_hidden) # przejście przez warstwę rekurencyjną
        attn_weights = self.attn(rnn_output, encoder_outputs) # obliczamy wagi dla mechanizmu uwagi
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1)) 
        # bmm to batched matrix multiplication, zwyczajnie mnożymy tutaj wagi przez stany ukryte
        rnn_output = rnn_output.squeeze(0)# usunięcie zbędnego wymiaru
        context = context.squeeze(1) # usunięcie zbędnego wymiaru
        concat_input = torch.cat((rnn_output, context), 1) # konkatenacja wyjścia z GRU, i wyniku mechanizmu uwagi
        concat_output = torch.tanh(self.concat(concat_input)) # przejście przez warstwę gęstą z funkcją aktywacji tanh
        output = self.out(concat_output) # klasyfikacja - przewidywanie następnych słów
        output = self.softmax(output)
        return output, hidden # zwracamy przewidywane słowo, i stan ukryty dekodera


#### Definicja funkcji straty
Potrzebujemy zdefiniować funkcje straty, aby móc maskować stratę na tokenach, które służą paddingowi.

In [ ]:
def maskNLLLoss(inp, target, mask):
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1)) # obliczanie entropii krzyżowej 
    #(- logarytm z prawdopodobieństwa zwróconego dla wartości docelowej)
    loss = crossEntropy.masked_select(mask).mean() # maskowanie straty (względem paddingu)
    loss = loss.to(DEVICE)
    return loss

#### Definicja funkcji do treningu i ewaluacji

In [ ]:
def train_on_batch(X, X_lens, Y_lens, Y, mask, encoder, decoder, encoder_optimizer, decoder_optimizer, teacher_forcing_ratio, grad_clip):

    # Zerowanie gradientów
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    X = X.to(DEVICE)
    Y = Y.to(DEVICE)
    mask = mask.to(DEVICE)
    # Tensory reprezentujące długośc sekwencji (dla pakowania) muszą znajdować się zawsze na CPU
    X_lens = X_lens.to("cpu")

    loss = 0

    encoder_outputs, encoder_hidden = encoder(X, X_lens)# przejście przez enkoder

    # Wejście do dekodera w kroku zerowym - tokeny SOS
    decoder_input = torch.LongTensor([[EN_SOS_TOKEN for _ in range(BATCH_SIZE)]]).to(DEVICE)

    # W kroku zero, stanem ukrytym dekodera, jest stan ukryty z enkodera
    decoder_hidden = encoder_hidden[:decoder.n_layers] # 

    use_teacher_forcing = random.random() < teacher_forcing_ratio# losowanie czy w tym przykładzie użyć teacher forcing

    max_target_len = max(Y_lens)
    if use_teacher_forcing: # teacher forcing
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Za następny input, uznajemy wzorcowy output z tego kroku
            decoder_input = Y[t].view(1, -1)
            # obliczenie straty
            mask_loss = maskNLLLoss(decoder_output, Y[t], mask[t])
            loss += mask_loss

    else: # sieć radzi sobie sama
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # sieć działa sama, inputem w kroku kolejnym jest output z obecnego kroku
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(BATCH_SIZE)]])
            decoder_input = decoder_input.to(DEVICE)
            mask_loss = maskNLLLoss(decoder_output, Y[t], mask[t])
            loss += mask_loss

    loss.backward()

    # Obcinanie gradientów
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), grad_clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), grad_clip)
    # Aktualizacja wag
    encoder_optimizer.step()
    decoder_optimizer.step()
    return loss.item()


def test_on_batch(X, X_lens, Y_lens, Y, mask, encoder, decoder, teacher_forcing_ratio=0.5):
    X = X.to(DEVICE)
    Y = Y.to(DEVICE)
    mask = mask.to(DEVICE)
    X_lens = X_lens.to("cpu")

    loss = 0

    encoder_outputs, encoder_hidden = encoder(X, X_lens)
    decoder_input = torch.LongTensor([[EN_SOS_TOKEN for _ in range(BATCH_SIZE)]])
    decoder_input = decoder_input.to(DEVICE)
    decoder_hidden = encoder_hidden[:decoder.n_layers]
    use_teacher_forcing = random.random() < teacher_forcing_ratio
    max_target_len = max(Y_lens)
    decoder_outputs = []
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_input = Y[t].view(1, -1)
            mask_loss = maskNLLLoss(decoder_output, Y[t], mask[t])
            loss += mask_loss
            decoder_outputs.append(decoder_output.unsqueeze(0))
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(BATCH_SIZE)]])
            decoder_input = decoder_input.to(DEVICE)
            mask_loss = maskNLLLoss(decoder_output, Y[t], mask[t])
            loss += mask_loss
            decoder_outputs.append(decoder_output.unsqueeze(0))
    decoder_outputs = torch.cat(decoder_outputs)
    return loss.item(), decoder_outputs

#### Przykładowe przejście przez sieć

Stworzenie sieci i przygotowanie danych

In [ ]:
import torch

attn_model = 'general'
hidden_size = 600
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.5
pl_voc_size = len(pl_voc) + 2
en_voc_size = len(en_voc) + 3
MAXLEN = 10 + 1
BATCH_SIZE = 8


encoder = EncoderRNN(hidden_size, pl_voc_size, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, hidden_size, en_voc_size, decoder_n_layers, dropout)
attention = decoder.attn

examples = data[0:8]
batch = examples_to_batch(examples, MAXLEN)
X, Y, X_lens, Y_lens, Y_mask = batch

Przejście przez enkoder

In [ ]:
# ENKODER
print("Kształt wejścia: ", X.shape) # liczba tokenów, numer sekwencji w batchu
print(X)# sekwencje są opaddowane i posortowane względem długości
embedded = encoder.embedding(X) # konwersja indeksów słów w wektory
print("Kształt po zanurzeniu: ", embedded.shape) # liczba tokenów, numer sekwencji w batchu, liczba cech
packed = nn.utils.rnn.pack_padded_sequence(embedded, X_lens)# pakowanie batcha danych dla warstwy GRU
outputs, encoder_hidden = encoder.gru(packed, None) # przekazujemy spakowany input, bez stanu zerowego
outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs) # odpakowywanie wyjścia z warstwy GRU
print("Kształt wyjścia z warstwy rekurencyjnej: ", outputs.shape) # liczba tokenów, numer sekwencji w batchu, liczba cech (z obu kierunków)
left_to_right = outputs[:, :, :encoder.hidden_size]
right_to_left = outputs[:, : ,encoder.hidden_size:]
encoder_outputs =  left_to_right + right_to_left  # sumujemy reprezentacje z obu kierunków warstwy rekurencyjnej
print("Kształt po agregacji z obu kierunków: ", encoder_outputs.shape) # liczba tokenów, numer sekwencji w batchu, liczba cech

Przejście przez dekoder

In [ ]:
# DEKODER
decoder_input = torch.LongTensor([[EN_SOS_TOKEN for _ in range(BATCH_SIZE)]]) # input w kroku zero - tokeny SOS
print("Kształt wejścia do dekodera (słowa angielskie): ", decoder_input.shape)# numer tokenu (podajemy po jednym tokenie), numer sekwencji w batchu
decoder_hidden = encoder_hidden[:decoder.n_layers] # stan ukryty w kroku zero - stan ukryty z obu warstw enkodera
print("Kształt stanu ukrytego dekodera: ", decoder_hidden.shape) # numer warstwy, numer w batchu, numer cechy

max_target_len = max(Y_lens) # iterujemy aż do końca najdłuższej sekwencji w Y (zdania najdłuższego po angielsku)
for t in range(max_target_len): # iterujemy po krokach czasowych
    # dekoder przechodzi przez cały batch słowo po słowie (i.e. zbiera najpierw wszystkie pierwsze słowa, potem wszystkie drugie itd.)
    embedded = decoder.embedding(decoder_input) # zanurzenie  słów
    print("Kształt zanurzonych angielskich słów: ", embedded.shape) # numer słowa, numer sekwencji w batchu, numer cechy
    embedded = decoder.embedding_dropout(embedded) # aplikacja dropoutu
    rnn_output, hidden = decoder.gru(embedded, decoder_hidden) # przejście przez warstwę rekurencyjną
    print("Kształt wyjścia z warstwy rekurencyjnej: ", rnn_output.shape) # numer słowa, numer sekwencji w batchu, numer cechy
    print("Kształt stanów ukrytych z warstw rekurencyjnych: ", hidden.shape) # numer warstwy, numer sekwencji w batchu, numer cechy

    # ATENCJA
    energy = attention.attn(encoder_outputs)
    attn_energies = torch.sum(rnn_output * energy, dim=2).t()
    print("Kształt wag (dla każdego kroku czasowego wejścia):", attn_energies.shape)
    normalized_attn = attention.softmax(attn_energies).unsqueeze(1) # normalizacja funkcją softmax

    # DEKODER Ciąg dalszy
    context = normalized_attn.bmm(encoder_outputs.transpose(0, 1)) # bmm to batched matrix multiplication, zwyczajnie mnożymy tutaj wagi przez stany ukryte
    rnn_output = rnn_output.squeeze(0)# usunięcie zbędnego wymiaru
    context = context.squeeze(1) # usunięcie zbędnego wymiaru
    print("Kształt wektora kontekstu uśrednionego przy pomocy wag: ", context.shape) # numer w batchu, numer cechy
    concat_input = torch.cat((rnn_output, context), 1) # konkatenacja wyjścia z GRU, i wyniku mechanizmu uwagi
    print("Kształt konkatenowanego wejścia do warstwy gęstej (stan ukryty dekodera + wektor kontekstu)", concat_input.shape) # numer w batchu, liczba cech
    concat_output = torch.tanh(decoder.concat(concat_input)) # przejście przez warstwę gęstą z funkcją aktywacji tanh
    print("Kszałt wejścia do klasyfikatora: ", concat_output.shape) # numer sekwencji w batchu, numer cechy
    output = decoder.out(concat_output) # klasyfikacja - przewidywanie następnych słów
    decoder_output = decoder.softmax(output)
    print("Kształt wyjścia z dekodera: ", decoder_output.shape) # numer w sekwencji, numer w słowniku

    _, topi = decoder_output.topk(1) # przewidziane słowo
    print("Przewidziane pierwsze słowa dla każdego ze zdań: ", [en_voc[i] for i in topi])
    decoder_input = torch.LongTensor([[topi[i][0] for i in range(BATCH_SIZE)]])
    print("Wyjście kierowane z powrotem do dekodera: ", decoder_input)
    # przesunięcie przewidzianego słowa jako input dla nastepnego kroku
    break # zobaczymy tylko jedno przejście


## Trening

In [ ]:
from torch import optim
import random
from tqdm.notebook import tqdm

DEVICE = torch.device("cuda:0")
learning_rate=0.0002
GRAD_CLIP = 1.0
decoder_learning_ratio = 5.0
TEACHER_FORCING_RATIO = 0.5
attn_model = 'general'
hidden_size = 600
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.5
pl_voc_size = len(pl_voc) + 2
en_voc_size = len(en_voc) + 3
MAXLEN = 10 + 1
BATCH_SIZE = 64
N_EPOCHS = 2
encoder = EncoderRNN(hidden_size, pl_voc_size, encoder_n_layers, dropout).to(DEVICE)
decoder = LuongAttnDecoderRNN(attn_model, hidden_size, en_voc_size, decoder_n_layers, dropout).to(DEVICE)
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
train_losses = []
dev_losses = []

train_data = data[:-5000]
dev_data = data[-5000:]
TRAIN_ITERS = len(train_data)//BATCH_SIZE
DEV_ITERS = len(dev_data)//BATCH_SIZE
LOSS_PRINT_INTERVAL = 2000


for epoch in range(N_EPOCHS):
    random.shuffle(train_data)
    print("Epoch no: ", epoch + 1)
    interval_losses = []
    encoder.train()
    decoder.train()

    for iter in tqdm(range(TRAIN_ITERS)):
        examples = train_data[iter*BATCH_SIZE:(iter+1)*BATCH_SIZE]
        batch = examples_to_batch(examples, MAXLEN)

        # Extract fields from batch
        X, Y, X_lens, Y_lens, Y_mask = batch

        loss = train_on_batch(X, X_lens, Y_lens, Y, Y_mask, encoder, decoder, encoder_optimizer, decoder_optimizer, TEACHER_FORCING_RATIO, GRAD_CLIP)
        interval_losses.append(loss)
        if iter % LOSS_PRINT_INTERVAL == 0:
            interval_loss = sum(interval_losses)/len(interval_losses)
            print("{} Interval loss: ".format(iter), interval_loss)
            interval_losses = []
    with torch.no_grad():
        dev_epoch_losses = []
        encoder.eval()
        decoder.eval()
        for iter in range(DEV_ITERS):
            examples = dev_data[iter*BATCH_SIZE:(iter+1)*BATCH_SIZE]
            batch = examples_to_batch(examples, MAXLEN)
            X, Y, X_lens, Y_lens, Y_mask = batch
            dev_loss, _ = test_on_batch(X, X_lens, Y_lens, Y, Y_mask, encoder, decoder)
            dev_epoch_losses.append(dev_loss)

    dev_epoch_loss = sum(dev_epoch_losses)/len(dev_epoch_losses)
    print("Dev loss: ", dev_epoch_loss)
    dev_losses.append(dev_epoch_loss)


#### Zapisywanie i wczytywanie modelu

In [ ]:
torch.save(encoder, "encoder.model")
torch.save(decoder, "decoder.model")

Tę komórkę należy wywołać, aby ominąć trening, i wczytać gotowy model

In [ ]:
encoder = torch.load("encoder.model")
decoder = torch.load("decoder.model")

#### Testowanie na zdaniach

In [ ]:
import re

DEVICE = torch.device("cuda:0")
MAXLEN = 10 + 1
BATCH_SIZE = 1

def test_on_sent(sentence):
    tokenizer = re.compile(r"[\w]+")
    tokens = [t for t in tokenizer.findall(sentence.lower()) if t in pl_voc]
    pl_ids = [[pl_voc.index(t) for t in tokens]]
    X_lens = torch.LongTensor([len(pl) for pl in pl_ids])
    pl_maxlen = max(X_lens)
    X = torch.LongTensor(pl_ids).transpose(1,0).to(DEVICE)
    encoder_outputs, encoder_hidden = encoder(X, X_lens)
    decoder_input = torch.LongTensor([[EN_SOS_TOKEN for _ in range(BATCH_SIZE)]]).to(DEVICE)
    decoder_hidden = encoder_hidden[:decoder.n_layers]
    max_target_len = MAXLEN
    decoder_outputs = []
    top_outs = []
    for t in range(max_target_len):
        decoder_output, decoder_hidden = decoder(
            decoder_input, decoder_hidden, encoder_outputs
        )
        _, topi = decoder_output.topk(1)
        top_outs.append(topi.item())
        decoder_input = torch.LongTensor([[topi[i][0] for i in range(BATCH_SIZE)]]).to(DEVICE)
        decoder_outputs.append(decoder_output.unsqueeze(0))
    decoder_outputs = torch.cat(decoder_outputs) 
    en_words = [en_voc[i] for i in top_outs if i!=EN_EOS_TOKEN]
    return en_words

translation = test_on_sent("Lubię czytać długie książki")
print(translation)